# Experiment 07

## Introduction

...

...

...

## Background

...

...

...

## Implementation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

from expt_utils import *

In [ ]:
OUT_DIR = make_out_dir('07')

In [ ]:
img = cv.imread("../dataset/frames/train/00012/00012_1560.jpg")
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

In [ ]:
edges = cv.Canny(img, 100, 200)

blurred = cv.GaussianBlur(img, (13, 13), 0)

sobel_x = cv.Sobel(img, cv.CV_64F, 1, 0, ksize=3)
sobel_y = cv.Sobel(img, cv.CV_64F, 0, 1, ksize=3)
sobel_combined = cv.magnitude(sobel_x, sobel_y)

log = cv.Laplacian(cv.GaussianBlur(img, (3, 3), 0), cv.CV_64F)

plt.figure(figsize=(10, 6), tight_layout=True)

plt.subplot(2, 2, 1), plt.axis('off'), plt.title('Canny')
plt.imshow(edges, cmap='gray')

plt.subplot(2, 2, 2), plt.axis('off'), plt.title('Gaussian')
plt.imshow(blurred, cmap='gray')

plt.subplot(2, 2, 3), plt.axis('off'), plt.title('Mexican Hat')
plt.imshow(sobel_combined, cmap='gray')

plt.subplot(2, 2, 4), plt.axis('off'), plt.title('Hanny')
plt.imshow(log, cmap='gray')

plt.savefig(f"{OUT_DIR}/1-some_filters.png",
            format='png', bbox_inches='tight')

In [ ]:
kernel_sizes = [3, 5, 7]
num_kernel = len(kernel_sizes)


def compute_metrics(edges):
    contours, _ = cv.findContours(
        edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    edge_density = len(contours)

    total_length = 0
    for contour in contours:
        total_length += cv.arcLength(contour, closed=True)
    edge_length = total_length

    return edge_density, edge_length


plt.figure(figsize=(5, 3 * num_kernel))

for i, kernel_size in enumerate(kernel_sizes):
    detected_edges = cv.Canny(img_gray, 100, 200, kernel_size)
    edge_density, edge_length = compute_metrics(detected_edges)
    print(
        f"Kernel size: {kernel_size}, Edge Density: {edge_density}, Edge Length: {edge_length}")

    plt.subplot(num_kernel, 1, i + 1),  plt.axis('off')
    plt.title(f"figure with size : {kernel_size}")
    plt.imshow(detected_edges, cmap='gray')

plt.savefig(f"{OUT_DIR}/2-multi_kernel_sizes.png",
            format='png', bbox_inches='tight')

## Conclusion

...

...

...